<a href="https://colab.research.google.com/github/eyaler/constrained/blob/main/density/ted_multilingual_counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install PyICU

import icu
from collections import Counter
from math import exp, log
import re


def clean(s):
  return re.sub('[\u200b-\u200f®©™♪♫\ufeff]', '', s).strip()


trans = str.maketrans('ς' + 'ךםןףץ' + '٩٨٧٦٥٤٣٢١٠' + '،' + '\xa0', 'σ' + 'כמנפצ' + '9876543210' + ',' + ' ')
print([(chr(k), chr(v)) for k, v in trans.items()])


def norm(s):
  return s.translate(trans).lower()


def perplexity(cnt):
  s = sum(cnt.values())
  return exp(-sum(log(c / s) * c for c in cnt.values()) / s)

[('ς', 'σ'), ('ך', 'כ'), ('ם', 'מ'), ('ן', 'נ'), ('ף', 'פ'), ('ץ', 'צ'), ('٩', '9'), ('٨', '8'), ('٧', '7'), ('٦', '6'), ('٥', '5'), ('٤', '4'), ('٣', '3'), ('٢', '2'), ('١', '1'), ('٠', '0'), ('،', ','), ('\xa0', ' ')]


# TED2020

    @inproceedings{reimers-2020-multilingual-sentence-bert,
      title = "Making Monolingual Sentence Embeddings Multilingual using Knowledge Distillation",
      author = "Reimers, Nils and Gurevych, Iryna",
      booktitle = "Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing",
      month = "11",
      year = "2020",
      publisher = "Association for Computational Linguistics",
      url = "https://arxiv.org/abs/2004.09813",
    }

https://arxiv.org/abs/2004.09813

https://huggingface.co/datasets/wecover/OPUS_TED2020

In [70]:
from datasets import load_dataset


ds = load_dataset('wecover/OPUS_TED2020', 'he')

Resolving data files:   0%|          | 0/2723 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2723 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2723 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/76 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/76 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/76 [00:00<?, ?it/s]

In [71]:
bi = {}

for _, rows in ds.items():
  for row in rows:
    laaz = (row['lang1'], clean(row['sentence1']))
    heb = (row['lang2'], clean(row['sentence2']))

    if laaz[0] == 'he':
      laaz, heb = heb, laaz
    assert heb[0] == 'he', row
    lang = laaz[0]
    if lang not in bi:
      bi[lang] = [0, 0, Counter(), Counter(), 0]
    bi[lang][0] += len(laaz[1])
    bi[lang][1] += len(heb[1])
    bi[lang][2].update(norm(laaz[1]))
    bi[lang][3].update(norm(heb[1]))
    bi[lang][4] += 1

ted_2020 = dict(sorted([(k, (icu.Locale(k).getDisplayName(), v[0] / v[1], len(v[2]), perplexity(v[2]), perplexity(v[3]), v[4])) for k, v in bi.items()], key=lambda x: (x[1][1], x[0])))

for code, (lang, rlen, symbols, perp, pheb, sents) in ted_2020.items():
  print(f'({code}) {lang}\t{round(rlen, 3):.3f} {symbols} {round(perp, 1):.1f} {round(perp / pheb * rlen, 3):.3f} ({sents})')

(ja) Japanese	0.545 2632 224.7 6.225 (25000)
(ko) Korean	0.698 1689 117.4 4.158 (25000)
(am) Amharic	0.786 279 63.5 2.500 (1023)
(la) Latin	0.995 30 18.2 0.953 (20)
(ar) Arabic	1.092 232 23.7 1.317 (25000)
(ps) Pashto	1.165 102 21.9 1.308 (962)
(kk) Kazakh	1.177 114 26.1 1.560 (9484)
(pl) Polish	1.183 110 25.5 1.532 (25000)
(th) Thai	1.194 148 41.8 2.544 (25000)
(sl) Slovenian	1.196 96 20.6 1.255 (25000)
(cs) Czech	1.200 102 27.0 1.655 (25000)
(as) Assamese	1.212 65 32.1 2.079 (41)
(sr) Serbian	1.216 112 29.6 1.832 (25000)
(mr) Marathi	1.216 159 30.8 1.910 (20287)
(sk) Slovak	1.217 102 26.2 1.620 (25000)
(bs) Bosnian	1.233 76 20.6 1.295 (11058)
(hr) Croatian	1.236 95 20.7 1.306 (25000)
(eo) Esperanto	1.236 77 20.1 1.250 (11079)
(uk) Ukrainian	1.240 119 26.5 1.680 (25000)
(mn) Mongolian	1.240 112 23.9 1.505 (23778)
(az) Azerbaijani	1.241 75 24.8 1.556 (11402)
(ne) Nepali	1.243 132 32.2 2.020 (4112)
(be) Belarusian	1.245 107 26.5 1.685 (10036)
(fi) Finnish	1.246 92 18.9 1.193 (25000)
(et

# TED-Parallel-Corpus

    @techreport{kulkarni:hal-04702210,
      TITLE = {{TED-Parallel-Corpus}},
      AUTHOR = {Kulkarni, Ajinkya},
      URL = {https://hal.science/hal-04702210},
      NUMBER = {none},
      INSTITUTION = {{CDAC ; Idiap Research Institure}},
      YEAR = {2015},
      MONTH = Jan,
      KEYWORDS = {Machine translation ; Neural machine translation NMT ; NLP ; Multilingual},
      PDF = {https://hal.science/hal-04702210v1/file/ted_parallel_corpus.pdf},
      HAL_ID = {hal-04702210},
      HAL_VERSION = {v1},
    }

https://hal.science/hal-04702210

https://github.com/ajinkyakulkarni14/How-I-Extracted-TED-talks-for-parallel-Corpus-

https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/

In [72]:
!wget https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/raw/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip -O Multi_lingual_Parallel_corpus_1.zip
!unzip -o Multi_lingual_Parallel_corpus_1.zip

--2025-12-08 13:26:52--  https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/raw/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip [following]
--2025-12-08 13:26:52--  https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

In [73]:
last = 0
sents = 0
seen = set()
multi = {}
counters = {}

with open(r'Multilingual_Parllel_corpus.txt', encoding='utf8') as f:
  for line in f.readlines():
    line = clean(line)
    if re.match(r'\d+:[a-z]{2}(-[a-z]{2})?:', line):
      num = int(line.split(':', 1)[0])
      if num != last:
        assert not seen or len(seen) == 9, (last, seen)
        seen = set()
        last = num
        sents += 1
      parts = line.split(':', 2)
      lang = parts[1]
      if lang in seen:
        continue
      seen.add(lang)
      if lang not in multi:
        multi[lang] = 0
        counters[lang] = Counter()
      text = parts[2]
    else:
      text = ' ' + line
    multi[lang] += len(text)
    counters[lang].update(norm(text))

print(f'{sents=}')
ted_para = dict(sorted([(k, (icu.Locale(k).getDisplayName(), v / multi['he'], len(counters[k]), perplexity(counters[k]))) for k, v in multi.items()], key=lambda x: (x[1][1], x[0])))
pheb_para = perplexity(counters['he'])
for code, (lang, rlen, symbols, perp) in ted_para.items():
  print(f'({code}) {lang}\t{round(rlen, 3):.3f} {symbols} {round(perp, 1):.1f} {round(perp / pheb_para * rlen, 3):.3f}')

sents=349049
(he) Hebrew	1.000 121 20.1 1.000
(ar) Arabic	1.104 266 24.2 1.323
(ru) Russian	1.322 145 25.3 1.660
(nl) Dutch	1.347 122 19.0 1.269
(es) Spanish	1.372 167 19.8 1.347
(pt-br) Portuguese (Brazil)	1.377 126 20.1 1.375
(it) Italian	1.399 119 19.2 1.332
(de) German	1.475 132 19.9 1.458
(fr) French	1.512 147 20.5 1.542


In [74]:
import json


exclude = ['la']  # Latin is just 20 sentences which are short Latin idioms

output = []
ted_2020 = {'he': ('Hebrew', 1.0)} | ted_2020
for code, (lang, rlen, *rest) in ted_2020.items():
  if code in exclude:
    continue
  d = dict(Code=code, Lang=lang, Length_2020=round(rlen, 3))
  if rest:
    symbols, perp, pheb, sents = rest
    d.update(dict(Symbols_2020=symbols, Perplexity_2020=round(perp, 1), Inefficiency_2020=round(perp / pheb * rlen, 3), Sentences_2020=sents))
  else:
    d.update(dict(Symbols_2020=None, Perplexity_2020=None, Inefficiency_2020=1.0, Sentences_2020=None))
  if code in ted_para:
    rlen, symbols, perp = ted_para[code][1:]
    d.update(dict(Length_Parallel=round(rlen, 3), Symbols_Parallel=symbols, Perplexity_Parallel=round(perp, 1), Inefficiency_Parallel=round(perp / pheb_para * rlen, 3)))
  output.append(d)
for code, (lang, rlen, symbols, perp) in ted_para.items():
  if code not in exclude and code not in ted_2020:
    output.append(dict(Code=code, Lang=lang, Length_Parallel=round(rlen, 3), Symbols_Parallel=symbols, Perplexity_Parallel=round(perp, 1), Inefficiency_Parallel=round(perp / pheb_para * rlen, 3)))
with open('density.json', 'w') as f:
  json.dump(output, f)